In [1]:
# Recipe - Pattern 2B - Persistent window (multiple reads using an event loop + updates data in window)
# https://www.pysimplegui.org/en/latest/cookbook/

## The link above should contain information about how to refresh the GUI so that the match generated data can be displayed

In [ ]:
import PySimpleGUI as sg
import csv
import random
import pandas as pd


# Creation of a dataframe containing the pictures of future matches

dict_pics = {'ladies':['lady_1.png', 'lady_2.png', 'lady_3.png', 'lady_4.png'],
            'gentlemen':['gentleman_1.png', 'gentleman_2.png', 'gentleman_3.png', 'gentleman_4.png']
            }

pics_df = pd.DataFrame(data = dict_pics)



# GUI

sg.theme('BrightColors')
sg.set_options(font=('Arial Bold', 16))


# Reading the .csv file with the generated match

filename = 'sample_match.csv' # please, replace with the actual file name from the file that has just been created

with open(filename, "r") as infile: 
            reader = csv.reader(infile)
            header_list = next(reader)
            data = list(reader)


            
# The interface is divided in the following areas areas: col0, col1, col2, col3, and col4


# col0 contains the title

col0 = sg.Column([[sg.Text("KLUST", text_color="#343434", font='Courier 40', pad = (400,0))]])



# col1 contains the user input that will be used to find a match

col1 = sg.Column([
    [sg.Frame('THIS IS ABOUT YOU', [
        [sg.Text(), sg.Column([
            [sg.Text('Age')],
            [sg.Input(key='user_age', enable_events=True, size=(10, 1))],
            [sg.Text('Gender')],
            [sg.Combo(['Female', 'Male'], default_value='', key='user_gender', enable_events=True)],
            [sg.Text('Orientation')],
            [sg.Combo(['Straight', 'Gay', 'Bisexual'], default_value='', key='user_orientation', enable_events=True)],
            [sg.Text('Education')],
            [sg.Combo(['High School', 'Bachelor', 'Master', 'PhD'], default_value='', key='user_education', enable_events=True)],
            [sg.Text('Do you smoke?')],
            [sg.Combo(['Yes', 'No', 'Occasionally'], default_value='', key='user_smokes', enable_events=True)],
            [sg.Text('Do you drink?')],
            [sg.Combo(['Yes', 'No', 'Occasionally'], default_value='', key='user_drinks', enable_events=True)],
            [sg.Text('Do you take drugs?')],
            [sg.Combo(['Yes', 'No', 'Occasionally'], default_value='', key='user_drugs', enable_events=True)],
            [sg.Text('Do you want kids?')],
            [sg.Combo(['Yes', 'No'], default_value='', key='user_kids', enable_events=True)],
            [sg.Text('Cats or dogs?')],
            [sg.Combo(['Cats', 'Dogs'], default_value='', key='user_pets', enable_events=True)],
        ], size=(250, 525), pad=(0, 0))]])] ], pad=(0, 0))



# col2 contains the button to gather the data from the user and pass it to a csv file

col2 = sg.Column([[sg.Button('Find Me a Match!', size=(15, 2), pad = (80,200))]],  size=(400, 550))



# col3 contains the AI-generated picture of the match

col3 = sg.Column([
    [
        sg.Frame('TIME TO INTRODUCE YOU TO YOUR MATCH!', [
            
        ])
    ],
    [
        sg.Image('?.png', expand_x=True, expand_y=True, key='-IMAGE-') # this key will be used later on to update the image
    ]
], pad=(0, 0))



# col4 contains general information about the match

col4 = sg.Column([[sg.Frame('INFORMATION ABOUT YOUR MATCH',
                             [[sg.Table(values=data,
                            headings=header_list,
                            max_col_width=25,
                            auto_size_columns=False,
                            justification='center',
                            num_rows=1, size = (3000,825), key='-MATCH_VALUES-') ]], pad=(0, 0), size = (4000,1000) )]])


# Layout of areas col1, col2, col3

layout = [[col0],[col1, col2, col3], [col4]]


# Title, definition and size of the GUI window

window = sg.Window('KLUST. Finding true love through K-means clusters.', layout, size=(1000, 725))



# Recording in a csv file the events (clicks) taking place in the GUI, that is, storing the user input in a csv file

while True:

    event, values = window.read()

    if event == sg.WINDOW_CLOSED:
        break

    
    elif event == 'Find Me a Match!': # When the "Find Me a Match!" button is pressed, the following code is executed

        # We first create a csv file with the user input data, which will be created to query the database
        
        with open('user_input.csv', 'w', newline='') as csvfile:
            
            writer = csv.writer(csvfile)
            
            values_keys = list(values.keys())
            values_keys.pop()
            
            values_values = list(values.values())
            values_values.pop()
            
            writer.writerow(values_keys)
            writer.writerow(values_values)      

        sg.popup('Looking for Love. \n\nPlease, be patient.', title='Important Message For You', font=('Courier', 52))

        
        # We assess whether the user will get a lady or a gentleman prompted
        
        user_input = pd.read_csv("user_input.csv")

        
        if user_input['user_orientation'].values[0] == 'straight' and user_input['user_gender'].values[0] == 'female':
            wanted_gender = 'male'
        elif user_input['user_orientation'].values[0] == 'straight' and user_input['user_gender'].values[0] == 'male':
            wanted_gender = 'female'
        elif user_input['user_orientation'].values[0] == 'gay' and user_input['user_gender'].values[0] == 'female':
            wanted_gender = 'female'
        elif user_input['user_orientation'].values[0] == 'gay' and user_input['user_gender'].values[0] == 'male':
            wanted_gender = 'male'
        elif user_input['user_orientation'].values[0] == 'bisexual':
            possible_genders = ['female','male']
            wanted_gender = random.choice(possible_genders)
        
        
        
        if wanted_gender == 'male':
            pic_filename = pics_df['gentlemen'].sample().values[0]
            print(pic_filename)
            
        elif wanted_gender == 'female':
            pic_filename = pics_df['ladies'].sample().values[0]
            print(pic_filename)
            

        
        
        
        
        # We now should query our database and look for the best matches and generate a csv with the features of the best match
        
        # The information from the csv will then be prompted in col4 in the GUI
        
        with open("sample_match2.csv", "r") as infile: 
                    reader = csv.reader(infile)
                    header_list = next(reader)
                    data = list(reader)

        window['-MATCH_VALUES-'].update(values = data)
        
        
        

        # The information from the csv file will then be used to generate an image of the match through DALL-E
        # Given that it was not possible for us to connect to DALL-E due to billing issues,
        # we decided to instead prompt pictures of Angela Merkel

        
    
        new_image = 'pics/' + pic_filename  # replace the image with the image of the generated match
        window['-IMAGE-'].update(filename = new_image)

    
    
        

window.close()

lady_4.png
